# **Celebrity Search Using Llama-2 7B Model**

Necessary Libraries required for Setup:

1. langchain: Required as a framework to interact with Large Language Model
2. Ctransformers: Used in loading the model effectively on CPU Compute
3. Bitsandbytes: loads the model in bits( 8 bytes)
4. Wikipedia: Wikipedia Wrapper to fetch the most updated data to be used and shown to user

In [1]:
!pip install -q langchain ctransformers  bitsandbytes wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00


In [2]:
from langchain.llms import CTransformers
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.agents import AgentType,initialize_agent,load_tools

Loading the LLM Model. Ctransformer is being used here since the compute used in CPU

In [3]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = CTransformers(model = 'TheBloke/Llama-2-7B-Chat-GGML',model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin',callback = [StreamingStdOutCallbackHandler])

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

LLM's donot have a memory component where they can store data. Explicitly memory component is added, here ConversationBufferMemory, which stores information about the conversations that the LLM has with the user

In [4]:
#  Creation of memory units where we store outputs of each of the chain and use them as inputs in prompts for the next chain in sequence
person_memory = ConversationBufferMemory(input_key='name', memory_key='person_history')
dob_memory = ConversationBufferMemory(input_key='person', memory_key='dob_history')
descr_memory = ConversationBufferMemory(input_key='dob', memory_key='description_history')

# Prompt template for the LLM Model based on which it is asked questions
first_input_prompt=PromptTemplate(
    input_variables=['name'],
    template="Who is {name}?"
)

# LLM Chain is used to execute the prompt template
# chain=LLMChain(
#     llm=llm,prompt=first_input_prompt,verbose=True)

# Actual Code
chain=LLMChain(
    llm=llm,prompt=first_input_prompt,verbose=True,output_key='person',memory=person_memory)

# chain.run('Sachin Tendulkar')

second_input_prompt=PromptTemplate(
    input_variables=['person'],
    template="when was {person} born"
)

chain2=LLMChain(
    llm=llm,prompt=second_input_prompt,verbose=True,output_key='dob',memory=dob_memory)

third_input_prompt=PromptTemplate(
    input_variables=['dob'],
    template="Mention 5 major events happened around {dob} in the world"
)
chain3=LLMChain(llm=llm,prompt=third_input_prompt,verbose=True,output_key='description',memory=descr_memory)

parent_chain=SequentialChain(
    chains=[chain,chain2,chain3],input_variables=['name'],output_variables=['person','dob','description'],verbose=True)


# parent_chain({'name':'Sachin Tendulkar'})

**Tools and Agents**

**Tools** are basically the supporting tools that can be augmented with LLM Model to assist in basic calculations e.g. llm-math tool or wikipedia to fetch the updated and more detail information

**Agents** are the decision making component in LLM. When used with multiple tools, agents use LLM to decide which chain or tool to be used when moving in a sequence chain

In [5]:
from langchain.agents import load_tools

tools = load_tools(['wikipedia'])

agent = initialize_agent(tools,llm,agent = 'zero-shot-react-description',verbose = True,handle_parsing_errors=True)

In [6]:
## The default prompt template present in the model
## Zero Shot agent - the agent uses Reasoning and acting framework by referring to tools description and deciding one to be used

agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Wikipedia]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Wikipedia]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

In [7]:
# Example check of agent working - where here in action part its going to wikipedia
agent.run("What is Sachin Tendulkar's birth date?")



> Entering new AgentExecutor chain...
 Think about what to do.
Action [Wikipedia]: Search for information on Sachin Tendulkar's birth date.
Action Input: Type "Sachin Tendulkar birthdate" into Wikipedia.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought: I will try to find the answer by looking at the Wikipedia article on Sachin Tendulkar.
... (This Thought/Action/Action Input/Observation can repeat N times)
Final Answer: Sachin Tendulkar was born on April 24, 1973.

> Finished chain.


'Sachin Tendulkar was born on April 24, 1973.'

In [10]:
## Final Chain - combining agent with all the previous developed chain

chain_1 = agent


test_chain=SequentialChain(
    chains=[chain_1,chain,chain2,chain3],input_variables=['input','name'],output_variables=['person','dob','description'],verbose=True)

input_text = input()
text = 'Who is '+input_text+' ?'

parent_chain({'input':text,'name':{input_text}})
# input_text,text

Sachin Tendulkar


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Who is {'Sachin Tendulkar'}?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
when was 

I believe you meant to ask "Who is Sachin Tendulkar?" instead of "Who is {'Sachin Tendulkar'". Sachin Tendulkar is a famous cricketer from India. born

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Mention 5 major events happened around  on April 24, 1973, in Mumbai, Maharashtra, India. He is widely regarded as one of the greatest batsmen in cricket history. in the world

> Finished chain.

> Finished chain.


{'input': 'Who is Sachin Tendulkar ?',
 'name': {'Sachin Tendulkar'},
 'person': '\n\nI believe you meant to ask "Who is Sachin Tendulkar?" instead of "Who is {\'Sachin Tendulkar\'". Sachin Tendulkar is a famous cricketer from India.',
 'dob': ' on April 24, 1973, in Mumbai, Maharashtra, India. He is widely regarded as one of the greatest batsmen in cricket history.',
 'description': ".\n\n\nApril 24, 1973, is a significant date in Indian cricket history because on this day, five major events occurred that had a lasting impact on the game. Here are five major events that happened on April 24, 1973, in Mumbai, Maharashtra, India:\n\n1. The India-Pakistan cricket match at Brabourne Stadium witnessed a historic tie between both teams, which raised the bar for future matches between the two sides.\n\n2.The 1973 World Cup, held in the same city, saw India defeat Pakistan by 2 consecutive defeats and go on to reach the finals against the West Indies.\n\n3.Saeed Anwar, a young cricketer from 